In [9]:
import pandas as pd
import numpy as np
import torch

In [ ]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [18]:
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [3]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )

In [ ]:
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
train.head(5)

In [5]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from sentence_transformers import SentenceTransformer

# GPU 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Embedding Vector 추출 모델을 GPU로 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False).to(device)

In [11]:
grouped = train.groupby(["공종(중분류)", "사고객체(중분류)", "공사종류(중분류)", "인적사고"])

res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    
    if len(plan) < 2:  # 데이터가 1개 이하라면 유사도 계산 불가
        continue  

    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    
    # 가장 평균 유사도가 높은 문장을 선택
    best_idx = similarity.mean(axis=1).argmax()
    cosine_res += similarity[best_idx].tolist()
    res[name] = plan.iloc[best_idx]

arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")


 97%|█████████▋| 8010/8262 [03:09<00:05, 42.26it/s] 

Range 0.0 - 0.1: 0개
Range 0.1 - 0.2: 21개
Range 0.2 - 0.3: 163개
Range 0.3 - 0.4: 556개
Range 0.4 - 0.5: 1542개
Range 0.5 - 0.6: 3263개
Range 0.6 - 0.7: 4839개
Range 0.7 - 0.8: 3440개
Range 0.8 - 0.9: 763개
Range 0.9 - 1.0: 1874개


In [12]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [17]:
for i in range(len(test)):
    key = tuple(test.loc[i, ["공종(중분류)", "사고객체(중분류)", "공사종류(중분류)", "인적사고"]])
    
    if key in res:  # res에 해당 키가 존재하는 경우
        sample.loc[i, "재발방지대책 및 향후조치계획"] = res[key]
    
    if key in res_v:  # res_v에도 해당 키가 존재하는 경우
        sample.iloc[i, 2:] = res_v[key]


In [21]:
sample.to_csv(base_path+"/v1.csv", index=False, encoding='utf-8-sig')

In [1]:
res

NameError: name 'res' is not defined